<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class8-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 8.1**

**Announcements**
* No new lab this week, but...
* Code for your Final Project will be due Nov. 21
  * That's *before* you leave for Thanksgiving.




# Interactive Matplotlib (not optional today)

Install `ipympl` for  Matplotlib


In [ ]:
!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
%matplotlib widget

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
The usual modules...

In [64]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal
import numpy.fft as fft
import pickle
import matplotlib.patches as patches
import matplotlib.colors
import matplotlib.cm

rc('animation', html='jshtml')
rc('animation', embed_limit=2**30)

path = '/content/drive/My Drive/eces435-work/class8.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'ub40'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helper functions

`myPlot()`

A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [71]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`myPlotFFT()`

Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [72]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, y_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim < 0:
      plt.ylim(y_lim, 0)
    elif y_lim > 0:
      plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`mySpectrogram()`

A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [73]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.stft(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 4*np.abs(Sxx) + 1e-15
  S_dBFS = 20*np.log10(S_mag)
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  # return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`zplane()` New function to plot system (filter) information in the $z$-plane.



In [79]:
def zplane(zeros=[], poles=[], xy_lim=[-1.5,1.5]):
  fig = plt.figure(figsize=(6,6))
  ax = plt.axes()

  # Create radial coordinates as for unit circle contour
  N_pts = 200
  w = np.linspace(0, 2*np.pi, N_pts)
  z_uc = np.exp(1j*w)
  re_uc = np.real(z_uc)
  im_uc = np.imag(z_uc)

  # Draw unit circle and set axis limits
  ax.plot(re_uc, im_uc, 'k:', linewidth=1)
  ax.set_ylim(xy_lim)
  ax.set_xlim(xy_lim)

  # Plot zeros (o) and poles (x)
  re_z = np.real(zeros)
  im_z = np.imag(zeros)
  ax.plot(re_z, im_z, 'bo', markersize=15, fillstyle='none')
  re_p = np.real(poles)
  im_p = np.imag(poles)
  ax.plot(re_p, im_p, 'bx', markersize=13, fillstyle='none')
  
  # X-Y axis labels
  ax.set_xlabel('Real')
  ax.set_ylabel('Imag')

`freqz3D()` A function to visualize the full response across the $z$-plane.

Lots of code here, most of it is to handle the graphics.

In [74]:
def freqz3D(B=[1], A=[1], azim=-60,
            x_lim=(-1.5,1.5), y_lim=(-1.5,1.5), z_lim=(-30,10),
            x_ticks=(-1,0,1), y_ticks=(-1,0,1),
            fig_size=(14,8), lw=2, xy_spacing=0.005, init_elev=45,
            draw_uc=True, draw_pz=True, draw_contour=True,
            hide_ax=False, hide_z=True, show_fig=True):

  fig = plt.figure(figsize=(14,8))
  ax = plt.axes(projection='3d') 
  z_dB_min = z_lim[0]

  if show_fig==False:
    plt.close()

  # Make cartesian grid in z-plane for freqz surface
  X = np.arange(-1.5, 1.501, xy_spacing) # dB requires different spacing
  Y = np.arange(-1.5, 1.501, xy_spacing) # dB spacing
  X, Y = np.meshgrid(X, Y)
  R = np.sqrt(X**2 + Y**2)
  z = X + Y*1j

  # Create radial coordinates as for unit circle contour(s)
  w = np.linspace(0, 2*np.pi, 200)
  z_c = np.exp(1j*w)
  X_c = np.real(z_c)
  Y_c = np.imag(z_c)

  # If transfer function is provided, plot the freqz surface
  if len(B) > 0:
    Bz =  B[0] * np.ones(np.shape(z))
    Bz_c = B[0] * np.ones(len(z_c))
    for n in range(1,len(B)):
      Bz = Bz + np.divide( B[n], z**n )
      Bz_c = Bz_c + np.divide( B[n], z_c**n )

    if len(A) > 0:
      Az = A[0] * np.ones(np.shape(z))
      Az_c = A[0] * np.ones(len(z_c))
      for n in range(1,len(A)):
        Az = Az + np.divide( A[n], z**n )
        Az_c = Az_c + np.divide( A[n], z_c**n )
    else:
      Az = 1
      Az_c = 1

    Hz = Bz / Az
    Hz_c = Bz_c / Az_c
    Z = np.abs(Hz)
    Z_c = np.abs(Hz_c)
    Z_dB = 20 * np.log10(Z)
    Z_c_dB = 20 * np.log10(Z_c)

    # Plot the freqz surface
    surf = ax.plot_surface(X, Y, Z_dB, color='lightgoldenrodyellow', alpha=0.4, linewidth=0 )
    
  Z_proj = np.zeros(len(w))
  Z_proj_dB = Z_proj + z_lim[0] # include dB offset, depends on z-axis lower limit

  # Draw unit circle at bottom face of z-plane
  if draw_uc:
    ax.plot3D(X_c, Y_c, Z_proj_dB, 'w',linewidth=1)

  # Draw freqz contour on unit circle
  an_object, = ax.plot3D(X_c, Y_c, Z_c_dB, 'y',linewidth=lw)
  if draw_contour == False:
    #Z_c_dB += 200 # Hopefully, +200dB is out of view
    an_object.set_linewidth(0.)

  # Create placeholder animation object for frequency marker
  an_object2, = ax.plot3D([0], [0], [-60], 'w:',linewidth=1)

  if draw_pz:
    if len(B) > 0:
      # Plot pole-zero markers
      zeros,poles,k = signal.tf2zpk(B, A)
      x_z = np.real(zeros)
      y_z = np.imag(zeros)
      x_p = np.real(poles)
      y_p = np.imag(poles)
      z_object, = ax.plot3D(x_z, y_z, z_dB_min*np.ones(len(x_z)), 'wo', markersize=15, fillstyle='none')
      p_object, = ax.plot3D(x_p, y_p, z_dB_min*np.ones(len(x_p)), 'wx', markersize=13, fillstyle='none')
  else:
    # Create empty objects
    z_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)
    p_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)


  # Set camera view angle
  ax.elev = init_elev
  ax.azim = azim

  # make the panes transparent
  ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.05))

  # make the grid lines transparent
  ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

  ax.zaxis.set_alpha(0.5)

  ax.set_zlim(z_lim) #dB
  ax.set_xlim(x_lim)
  ax.set_ylim(y_lim)

  if hide_ax:
    ax.set_axis_off()
  else:
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    ax.tick_params(colors='white')
    ax.tick_params(length=10,width=2,labelsize=20)
    for tick in ax.get_xticklabels():
        tick.set_fontname("Liberation Serif")

    for tick in ax.get_yticklabels():
        tick.set_fontname("Liberation Serif")

    fig.patch.set_facecolor('#111111')    # I think this makes the background dark, for preview in Colab

    #ax.set_frame_on(False)
    ax.set_facecolor('#212121')
    ax.get_xaxis().line.set_color('w')
    ax.get_yaxis().line.set_color('w')

    # X-Y axis labels
    ax.set_xlabel('Re', color='w', size=24, fontname='Liberation Serif')
    ax.xaxis.labelpad = 20
    ax.set_ylabel('Im', color='w', size=24, fontname='Liberation Serif')
    ax.yaxis.labelpad = 20

    if hide_z:
      ax.zaxis.line.set_lw(0.)
      ax.set_zticks([])

  fig.tight_layout()

#  if show_fig:
#    plt.show()

  return fig #, ax, an_object, an_object2, surf, p_object, z_object,

# The z-Transform

Just like the Fourier Transform, but with *additional information*.

* Where: $z = r \cdot e^{j\omega}$


$X(z) = \sum_n x[n] z^{-n} = \sum_n x[n] ( r \cdot e^{j\omega})^{-n} = \sum_n r^{-n} x[n] e^{-j\omega n}$


## Using the $z$-Transform

If you have a difference equation (linear, with constant coefficients)
* $y[n] + a_1 y[n-1] + ⋯ + a_M y[n-M] = b_0 x[n] + ⋯ + b_N x[n-N]$



### Take the $z$-Transform, which converts each delay into powers of $z^{-1}$:
* $Y(z) (1 + a_1 z^{-1} + ⋯ + a_M z^{-M}) = X(z) (b_0 + b_1 z^{-1} + ⋯ + b_N z^{-N})$

This makes it easy to derive the *transfer function*, $H(z)$:
* $H(z) = \frac{Y(z)}{X(z)} = \frac{b_0 + b_1 z^{-1} + ⋯ + b_N z^{-N}}{1 + a_1 z^{-1}+ ⋯ + a_M z^{-M}}$

The numerator and denominator can be factored to find the poles and zeros.

## IIR filters

* *IIR filters always incorporate feedback:* The output $y[n]$ depends on delayed samples of the output $y[n-k]$.
* This means that $H(z)$ has a denominator that is *not 1*.
  * This means that there are *poles* that will affect the ouput of the system.

Be careful:
* Due to feedback, IIR filters can easily be unstable (even when the input $x[n]$ is finite, the output $y[n]$ goes to infinity).

In [ ]:
Z = [1]

freq = 0.5
mag = 0.9

P = [mag*np.exp(1j*freq), mag*np.exp(-1j*freq)]

zplane(Z,P)

In [ ]:
B, A = signal.zpk2tf(Z, P, 1.)
w, H = signal.freqz(B, A)
w = w * fs44 / (2*np.pi)

H_dB = 20*np.log10(np.abs(H))
fig = plt.figure(figsize=(14,8))
plt.subplot(2,1,1)
plt.plot(w, H_dB)
plt.ylabel('Magnitude (dB)')
plt.subplot(2,1,2)
plt.plot(w, np.angle(H))
plt.ylabel('Phase (rad)')
plt.xlabel('Frequency (Hz)')

## Danger: avoid unstable filters

* Keep your poles *inside* the unit circle!

In [ ]:
d_n = np.zeros(1000)
d_n[0] = 1.

P_u = [1.2*np.exp(1j*0.5), 1.2*np.exp(-1j*0.5)]
zplane([], P_u)

In [ ]:
B_u, A_u = signal.zpk2tf([], P_u, 1.)

h2_iir = signal.lfilter(B_u,A_u,d_n)
plt.plot(h2_iir)

## Filter design by placing poles and zeros

In [ ]:
fs = 44100

p_mags = np.array([0.9, 0.95, 0.9, 0.9])
p_freqs = np.array([100, 1000, 5000, 10000])
p_angles = p_freqs * 2 * np.pi / (fs)

z_mags = np.array([0.5, 0.95, 0.95])
z_freqs = np.array([0, 2500, 15000])
z_angles = z_freqs * 2 * np.pi / (fs)

P1 = np.multiply(p_mags, np.exp(1j*p_angles))
P2 = np.multiply(p_mags, np.exp(-1j*p_angles))
P = np.append(P1,P2)

Z1 = np.multiply(z_mags, np.exp(1j*z_angles))
Z2 = np.multiply(z_mags, np.exp(-1j*z_angles))
Z = np.append(Z1,Z2)

zplane(Z,P)

In [ ]:
B, A = signal.zpk2tf([1], P, .2)
w, H = signal.freqz(B, A)
H_dB = 20*np.log10(np.abs(H))
fig = plt.figure(figsize=(14,8))
plt.subplot(2,1,1)
plt.plot(w, H_dB)
plt.subplot(2,1,2)
plt.plot(w, np.unwrap(np.angle(H)))

In [ ]:
fig4 = freqz3D(B,A)

## Interactive Pole-Zero interface

In [ ]:
# A mesh grid for the complex plane
x = np.linspace(-1.5, 1.5, num=1000)
y = 1j * np.linspace(-1.5, 1.5, num=1000)
z = np.add.outer(x, y).T

fs = 44100

# Initially start with no poles or zeros
zeros = []
poles = []


# Evaluate transfer function on the grid
def compute_H(z, zeros, poles):

    H = np.ones_like(z)
    for _z in np.asarray(zeros):
        H *= (z - _z)
    
    for _p in np.asarray(poles):
        H /= (z - _p)
        
    return H

H = compute_H(z, zeros, poles)

# Make a mosaic grid
fig = plt.figure(constrained_layout=True, figsize=(12, 8))
axd = fig.subplot_mosaic(
"""
ZT
FI
"""
)
axpz = axd['Z']
axtf = axd['T']
axf = axd['F']
axi = axd['I']
axpz.sharex(axtf)
axpz.sharey(axtf)

# Scaffold the pole-zero panel
axpz.set_aspect('equal')
axpz.grid(True, zorder=-10)
axpz.set(xlabel='Real', ylabel='Imaginary', title='Pole-Zero plot')
circ = patches.Ellipse((0, 0), 2, 2, edgecolor='k', linewidth=3, fill=False, zorder=1)

axpz.add_patch(circ)

zero_scat = axpz.scatter(np.asarray(zeros).real, np.asarray(zeros).imag, marker='o', facecolor='none', 
                         edgecolor='g', linewidth=3, zorder=2, s=50)
pole_scat = axpz.scatter(np.asarray(poles).real, np.asarray(poles).imag, marker='x', color='r', zorder=2, s=50)
axpz.set(xlim=[x.min(), x.max()], ylim=[x.min(), x.max()])


# Scaffold the frequency response panel
freq, h = scipy.signal.freqz_zpk(zeros, poles, 1, fs=fs)
#frcurve,  = axf.plot(freq, 20 * np.log10(np.abs(h) + 1e-12), linewidth=3)
h_db = 20 * np.log10(np.abs(h) + 1e-12)
frcurve = axf.scatter(freq, h_db, c=h_db, cmap='twilight_shifted', vmin=-80, vmax=80, marker='o', edgecolors='face', linewidth=2, s=5)
axf.set(title='Frequency response', xlabel='Frequency [Hz]', ylabel='$|H(z)|$ [dB]')
axf.axis('tight')
axf.set(ylim=[-80, 80], xlim=[0, fs/2])
axf.grid(True, zorder=-10)

Bz, Az = signal.zpk2tf(zeros, poles, 1)
d_n = np.zeros(64)
d_n[0] = 1
ir = signal.lfilter(Bz, Az, d_n)
ir_plot, = axi.plot(ir,'-o',markersize=2)
axi.set(xlabel='n', title='Impulse response')

# And the transfer function  plot
tfmesh = axtf.pcolormesh(x, x, 20 * np.log10(np.abs(H) + 1e-12), shading='nearest', cmap='twilight_shifted')
tfmesh.set_clim([-80, 80])
axtf.set(xlabel='Real', title='Transfer function magnitude')
circ = patches.Ellipse((0, 0), 2, 2, edgecolor='k', linewidth=2, fill=False, zorder=1, alpha=0.5)

axtf.add_patch(circ)
axtf.set_aspect('equal')
fig.colorbar(tfmesh, label='$|H(z)|$ [dB]', ax=axtf)

# Set up the interaction
def add_or_remove_point(event):
    global zeros
    global poles
      
    # Get the coordinates of the new point   
    new_point = event.xdata + 1j * event.ydata
    
    # Button 1 = add a zero
    if event.inaxes in (axtf, axpz):
        
        if event.button == 1:
            zeros.append(new_point)
            if event.ydata != 0:
                zeros.append(np.conj(new_point))

        # Button 3 = add a pole
        elif event.button == 3:
            poles.append(new_point)
            if event.ydata != 0:
                poles.append(np.conj(new_point))
        
    # Button 2 is reset
    if event.button == 2:
        zeros = []
        poles = []
        
    Z = np.asarray(zeros)
    zero_scat.set_offsets(np.vstack([Z.real, Z.imag]).T)
    
    P = np.asarray(poles)
    pole_scat.set_offsets(np.vstack([P.real, P.imag]).T)
        
    # Update the transfer function
    H = compute_H(z, zeros, poles)
    H_db = 20 * np.log10(np.abs(H) + 1e-12).ravel()
    tfmesh.set_array(H_db)
    vmax = max(40, max(np.abs(H_db)))
    tfmesh.set_clim([-vmax, vmax])
    
    # Update the frequency response curve
    freq, h = scipy.signal.freqz_zpk(zeros, poles, 1, fs=44100)
    h_db = 20 * np.log10(np.abs(h) + 1e-12)
    
    frcurve.set_offsets(np.vstack([freq, h_db]).T)
    
    n = matplotlib.colors.Normalize(vmin = -vmax, vmax = vmax)
    m = matplotlib.cm.ScalarMappable(norm=n, cmap=matplotlib.cm.twilight_shifted)
    frcurve.set_color(m.to_rgba(h_db))    
    frcurve.set_clim(vmin=-vmax, vmax=vmax)
        
    axf.set(ylim=[-vmax, vmax])

    d_n = np.zeros(64)
    d_n[0] = 1
    Bz,Az = signal.zpk2tf(zeros,poles,1)
    ir = signal.lfilter(Bz,Az,d_n)
    ir_plot.set_ydata(ir)
    axi.set_ylim([min(ir),max(ir)])

    plt.draw()
    
fig.canvas.mpl_connect('button_press_event', add_or_remove_point);

### Testing stuff (doesn't work)

In [ ]:
def freqz3D(input_ax, zeros, poles, azim=-60,
            x_lim=(-1.5,1.5), y_lim=(-1.5,1.5), z_lim=(-30,10),
            x_ticks=(-1,0,1), y_ticks=(-1,0,1),
            lw=2, xy_spacing=0.005, init_elev=45,
            draw_uc=True, draw_pz=True, draw_contour=True,
            hide_ax=False, hide_z=True, show_fig=True):

  ax = input_ax
  
  print(ax.collections)
  print(ax.lines)
  print(ax.spines)

  print(ax.get_children())
  # Remove previous surface plot
  ax.collections[0].remove()
  print(len(ax.lines))
  for n in range(min(2, len(ax.lines))):
    ax.lines[n].remove()

  z_dB_min = z_lim[0]

  # Make cartesian grid in z-plane for freqz surface
  X = np.arange(-1.5, 1.501, xy_spacing) # dB requires different spacing
  Y = np.arange(-1.5, 1.501, xy_spacing) # dB spacing
  X, Y = np.meshgrid(X, Y)
  R = np.sqrt(X**2 + Y**2)
  z = X + Y*1j

  # Create radial coordinates as for unit circle contour(s)
  w = np.linspace(0, 2*np.pi, 200)
  z_c = np.exp(1j*w)
  X_c = np.real(z_c)
  Y_c = np.imag(z_c)

  B,A = signal.zpk2tf(zeros, poles, 1.)

  # If transfer function is provided, plot the freqz surface
  if len(B) > 0:
    Bz =  B[0] * np.ones(np.shape(z))
    Bz_c = B[0] * np.ones(len(z_c))
    for n in range(1,len(B)):
      Bz = Bz + np.divide( B[n], z**n )
      Bz_c = Bz_c + np.divide( B[n], z_c**n )

    if len(A) > 0:
      Az = A[0] * np.ones(np.shape(z))
      Az_c = A[0] * np.ones(len(z_c))
      for n in range(1,len(A)):
        Az = Az + np.divide( A[n], z**n )
        Az_c = Az_c + np.divide( A[n], z_c**n )
    else:
      Az = 1
      Az_c = 1

    Hz = Bz / Az
    Hz_c = Bz_c / Az_c
    Z = np.abs(Hz)
    Z_c = np.abs(Hz_c)
    Z_dB = 20 * np.log10(Z)
    Z_c_dB = 20 * np.log10(Z_c)

    # Plot the freqz surface
    surf = ax.plot_surface(X, Y, Z_dB, color='lightgoldenrodyellow', alpha=0.4, linewidth=0 )
    ax.collections[0] = surf
    
  Z_proj = np.zeros(len(w))
  Z_proj_dB = Z_proj + z_lim[0] # include dB offset, depends on z-axis lower limit

  # Draw unit circle at bottom face of z-plane
  if draw_uc:
    ax.plot3D(X_c, Y_c, Z_proj_dB, 'w',linewidth=1)

  # Draw freqz contour on unit circle
  an_object, = ax.plot3D(X_c, Y_c, Z_c_dB, 'y',linewidth=lw)
  if draw_contour == False:
    #Z_c_dB += 200 # Hopefully, +200dB is out of view
    an_object.set_linewidth(0.)

  # Create placeholder animation object for frequency marker
  an_object2, = ax.plot3D([0], [0], [-60], 'w:',linewidth=1)

  if draw_pz:
    if len(B) > 0:
      # Plot pole-zero markers
      # zeros,poles,k = signal.tf2zpk(B, A)
      x_z = np.real(zeros)
      y_z = np.imag(zeros)
      x_p = np.real(poles)
      y_p = np.imag(poles)
      z_object, = ax.plot3D(x_z, y_z, z_dB_min*np.ones(len(x_z)), 'wo', markersize=15, fillstyle='none')
      p_object, = ax.plot3D(x_p, y_p, z_dB_min*np.ones(len(x_p)), 'wx', markersize=13, fillstyle='none')
  else:
    # Create empty objects
    z_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)
    p_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)


  # Set camera view angle
  ax.elev = init_elev
  ax.azim = azim

  # make the panes transparent
  ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.05))

  # make the grid lines transparent
  ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

  ax.zaxis.set_alpha(0.5)

  ax.set_zlim(z_lim) #dB
  ax.set_xlim(x_lim)
  ax.set_ylim(y_lim)

  if hide_ax:
    ax.set_axis_off()
  else:
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    ax.tick_params(colors='white')
    ax.tick_params(length=10,width=2,labelsize=20)
    for tick in ax.get_xticklabels():
        tick.set_fontname("Liberation Serif")

    for tick in ax.get_yticklabels():
        tick.set_fontname("Liberation Serif")

    fig.patch.set_facecolor('#111111')    # I think this makes the background dark, for preview in Colab

    #ax.set_frame_on(False)
    ax.set_facecolor('#212121')
    ax.get_xaxis().line.set_color('w')
    ax.get_yaxis().line.set_color('w')

    # X-Y axis labels
    ax.set_xlabel('Re', color='w', size=24, fontname='Liberation Serif')
    ax.xaxis.labelpad = 20
    ax.set_ylabel('Im', color='w', size=24, fontname='Liberation Serif')
    ax.yaxis.labelpad = 20

    if hide_z:
      ax.zaxis.line.set_lw(0.)
      ax.set_zticks([])

  fig.tight_layout()

#  if show_fig:
#    plt.show()

# return fig #, ax, an_object, an_object2, surf, p_object, z_object,

In [ ]:
# A mesh grid for the complex plane
x = np.linspace(-1.5, 1.5, num=1000)
y = 1j * np.linspace(-1.5, 1.5, num=1000)
z = np.add.outer(x, y).T

fs = 44100

# Initially start with no poles or zeros
zeros = []
poles = []


# Evaluate transfer function on the grid
def compute_H(z, zeros, poles):

    H = np.ones_like(z)
    for _z in np.asarray(zeros):
        H *= (z - _z)
    
    for _p in np.asarray(poles):
        H /= (z - _p)
        
    return H

H = compute_H(z, zeros, poles)
print(np.shape(H))

# Make a mosaic grid
fig = plt.figure(constrained_layout=True, figsize=(12, 8))
# axd = fig.subplot_mosaic(
# """
# ZT
# FF
# """
# )
# axpz = axd['Z']
# axtf = axd['T']
# axf = axd['F']
# axpz.sharex(axtf)
# axpz.sharey(axtf)

# fig,ax = plt.subplots(nrows = 2, ncols = 2)

# ax[0,1].remove()
# ax[0,1] = fig.add_subplot(2,2,2,projection='3d')
# axpz = ax[0,0]
# axtf = ax[0,1]
# axf = ax[1,0]

axpz = fig.add_subplot(2,2,1)
axtf = fig.add_subplot(2,2,2, projection='3d')
axf = fig.add_subplot(2,2,3)

# Scaffold the pole-zero panel
axpz.set_aspect('equal')
axpz.grid(True, zorder=-10)
axpz.set(xlabel='Real', ylabel='Imaginary', title='Pole-Zero plot')
circ = patches.Ellipse((0, 0), 2, 2, edgecolor='k', linewidth=3, fill=False, zorder=1)

axpz.add_patch(circ)

zero_scat = axpz.scatter(np.asarray(zeros).real, np.asarray(zeros).imag, marker='o', facecolor='none', 
                         edgecolor='g', linewidth=3, zorder=2, s=50)
pole_scat = axpz.scatter(np.asarray(poles).real, np.asarray(poles).imag, marker='x', color='r', zorder=2, s=50)
axpz.set(xlim=[x.min(), x.max()], ylim=[x.min(), x.max()])


# Scaffold the frequency response panel
freq, h = scipy.signal.freqz_zpk(zeros, poles, 1, fs=fs)
#frcurve,  = axf.plot(freq, 20 * np.log10(np.abs(h) + 1e-12), linewidth=3)
h_db = 20 * np.log10(np.abs(h) + 1e-12)
# frcurve = axf.scatter(freq, h_db, c=h_db, cmap='twilight_shifted', vmin=-80, vmax=80, marker='o', edgecolors='face', linewidth=10, s=20)
frcurve = axf.scatter(freq, h_db, c=h_db, cmap='twilight_shifted', vmin=-80, vmax=80, marker='o', edgecolors='b', linewidth=1, s=2)
axf.set(title='Frequency response', xlabel='Frequency [Hz]', ylabel='$|H(z)|$ [dB]')
# axf.axis('tight')
axf.set(ylim=[-80, 80], xlim=[0, fs/2])
axf.grid(True, zorder=-10)

# And the transfer function  plot
# tfmesh = axtf.pcolormesh(x, x, 20 * np.log10(np.abs(H) + 1e-12), shading='nearest', cmap='twilight_shifted')
tfmesh = axtf.plot_surface(x, x, 20 * np.log10(np.abs(H) + 1e-15), color='lightgoldenrodyellow', alpha=0.4, linewidth=0 )
print(id(axtf.collections[0]))
print(axtf.patches)
print(id(tfmesh))

# tfmesh.set_clim([-80, 80])
axtf.set(xlabel='Real', title='Transfer function magnitude')
# circ = patches.Ellipse((0, 0), 2, 2, edgecolor='k', linewidth=2, fill=False, zorder=1, alpha=0.5)
# axtf.add_patch(circ)
# axtf.set_aspect('equal')
# fig.colorbar(tfmesh, label='$|H(z)|$ [dB]', ax=axtf)

# Set up the interaction
def add_or_remove_point(event):
    global zeros
    global poles
       
    # Get the coordinates of the new point   
    new_point = event.xdata + 1j * event.ydata
    
    # Button 1 = add a zero
    if event.inaxes in (axtf, axpz):
        
        if event.button == 1:
            zeros.append(new_point)
            if event.ydata != 0:
                zeros.append(np.conj(new_point))

        # Button 3 = add a pole
        elif event.button == 3:
            poles.append(new_point)
            if event.ydata != 0:
                poles.append(np.conj(new_point))
        
    # Button 2 is reset
    if event.button == 2:
        zeros = []
        poles = []
        
    Z = np.asarray(zeros)
    zero_scat.set_offsets(np.vstack([Z.real, Z.imag]).T)
    
    P = np.asarray(poles)
    pole_scat.set_offsets(np.vstack([P.real, P.imag]).T)
        
    # Update the transfer function
    H = compute_H(z, zeros, poles)
    # H_db = 20 * np.log10(np.abs(H) + 1e-12).ravel()
    H_db = 20 * np.log10(np.abs(H) + 1e-15)
    print(np.shape(H_db))
    # print(tfmesh)
    # tfmesh.set_array(H_db)
    # ax[0,1].remove()
    # ax[0,1] = fig.add_subplot(2,2,2,projection='3d')
    # axtf.collections[0].remove()
    freqz3D(axtf, zeros, poles)
    # tfmesh_new = axtf.plot_surface(x, x, H_db, color='lightgoldenrodyellow', alpha=0.4, linewidth=0 )
    # axtf.collections[0] = tfmesh_new

    max_HdB = np.max( np.abs(H_db) )
    # vmax = max(40, max(np.abs(H_db)) )
    vmax = max(40, np.max(max_HdB) )
    # tfmesh.set_clim([-vmax, vmax])
    
    # Update the frequency response curve
    freq, h = scipy.signal.freqz_zpk(zeros, poles, 1, fs=44100)
    h_db = 20 * np.log10(np.abs(h) + 1e-12)
    
    frcurve.set_offsets(np.vstack([freq, h_db]).T)
    
    # n = matplotlib.colors.Normalize(vmin = -vmax, vmax = vmax)
    # m = matplotlib.cm.ScalarMappable(norm=n, cmap=matplotlib.cm.twilight_shifted)
    # frcurve.set_color(m.to_rgba(h_db))    
    # frcurve.set_clim(vmin=-vmax, vmax=vmax)
        
    axf.set(ylim=[-vmax, vmax])

    plt.draw()
    
fig.canvas.mpl_connect('button_press_event', add_or_remove_point);

# Returning to coding / compression

In [ ]:
x, fs44 = sf.read(path + 'u2.wav')
ipd.Audio(x, rate=fs44)

In [ ]:
f1, t1, X = signal.stft(x, fs44, nperseg=1024, noverlap=512, nfft=1024)

S_mag = np.abs(X) + 1e-15
S_dB = 20*np.log10(S_mag)

fig = plt.figure(figsize=(12,6))
plt.pcolormesh(t1, f1, S_dB)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')
# plt.show()

## Our basic encoder

In [ ]:
def encode(x):
  f1, t1, X = signal.stft(x, 44100, nperseg=1024, noverlap=512, nfft=1024)
  n_freqs, n_frames = np.shape(X)
  N_c = 60

  X_mag = np.abs(X)
  x_sortIdx = np.argsort(X_mag, axis=0)
  
  fft_idx = np.int16(x_sortIdx[-N_c:,:])
  fft_real = np.zeros([N_c, n_frames], dtype='float16')
  fft_imag = np.zeros([N_c, n_frames], dtype='float16')

  for n in range(n_frames):
    X_sort = X[fft_idx[:,n], n]
 
    fft_real[:,n] = np.real(X_sort)
    fft_imag[:,n] = np.imag(X_sort)

  y_compressed = [fft_idx, fft_real, fft_imag]
  return y_compressed

In [ ]:
def decode(y_compressed):
  fft_idx = y_compressed[0]
  fft_real = y_compressed[1]
  fft_imag = y_compressed[2]

  n_freqs, n_frames = np.shape(fft_idx)
  X_c = 1j * np.zeros([513, n_frames])

  for n in range(n_frames):
    X_c[fft_idx[:,n],n] = fft_real[:,n] + 1j*fft_imag[:,n]

  t_c, x_c = signal.istft(X_c, 44100, nperseg=1024, noverlap=512, nfft=1024)

  return x_c, X_c

In [ ]:
Y_c = encode(x)
x_c, X_c = decode(Y_c)
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(X_c)+1e-15))
ipd.Audio(x_c, rate=fs44)

## What if we reverse the process?

* Eliminate the highest magnitudes
* What's left?

In [ ]:
X_r = np.copy(X)
n_freqs, n_frames = np.shape(X_r)
x_idx = np.argsort(np.abs(X_r), axis=0)
N_c = 60

for n in range(n_frames):
  X_r[x_idx[-N_c:,n] , n] = 0

fig = plt.figure()
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(X_r)+1e-15))

In [ ]:
t, x_r = signal.istft(X_r, fs44, nperseg=1024, noverlap=512, nfft=1024)
ipd.Audio(x_r, rate=fs44)

In [ ]:
plt.plot(x_r)

## Modulated uniform noise

In [ ]:
e_n = np.sum(np.abs(X_r), axis=0)
plt.plot(e_n)

In [ ]:
x_n = np.zeros(len(x_c))
n_idx = 0

for n in range(n_frames-2):
  noise = e_n[n] * np.random.uniform(-1,1,1024)
  x_n[n_idx:n_idx+1024] += np.multiply(noise, np.hanning(1024))
  n_idx += 512

In [ ]:
plt.plot(x_n)

In [ ]:
ipd.Audio(x_n, rate=fs44)

## This is *Sinusoids plus Noise*

In [ ]:
ipd.Audio(x_c + 0.15*x_n, rate=fs44)

## A closer look at the frequency domain

In [ ]:
def fftAnim(input, num_frames=0, frame_rate=30, frame_size=2048, sample_rate=44100, n_fft=0, 
             fig_size=[16,4], x_lim=0, y_lim=[-120,5], x_ax=True, y_ax=True, lw=1, fmt='',
             save_figs=False, save_dir='waveAnim'):

  # The FFT animation function for iteration (called for each frame):
  def fftAnimFunc(f_num):
    win = np.hanning(frame_size)        # For now, we always use a Hanning (Hann) window    
    n_hop = int(sample_rate/frame_rate)
    n1 = int(n_hop*f_num)
    n2 = int(n_hop*f_num + frame_size)
    x_n = input[n1:n2]                  # Current frame of the input
    X_n = np.fft.fft(x_n * win, n_fft)  # The FFT of the current frame (windowed)
    N = len(X_n)
    f = np.arange(N) * sample_rate / N
    X_mag = 4*np.abs(X_n) / n_fft   # Frequency magnitude, normalized by length
                                    #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                    #    x2 because hanning window has 0.5 average
    X_mag += 1.0e-15                # Add a very small offset to avoid log(0) errors
    X_dBFS = 20*np.log10(X_mag)     # Freq. magnitude in dB full scale (dB FS):
                                    #    cos(w) -> 0 dBFS peak at w
    fftLine.set_data(f, X_dBFS)

    if save_figs==True:
        fftFig.savefig(save_dir + '/frame%04d.png' % n, dpi=100, transparent=True)
    
    return fftLine,

  # Extract/set default parameters
  if n_fft==0:
    n_fft = frame_size

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = sample_rate/2
    x_lim = [0, x_lim]

  # Set up the figure, the axis, and the plot element we want to animate
  fftFig = plt.figure(figsize=fig_size)
  ax = plt.axes(xlim=x_lim, ylim=y_lim)
  fftLine, = ax.plot([], [], fmt, linewidth=lw)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  fftFig.tight_layout()
  plt.close()             # Close the plot, so we don't get an extra figure added on

  frame_period_in_ms = 1000 / frame_rate
  if num_frames == 0:     # If num_frames==0 (default), use the full duration of the signal
    num_frames = np.floor( len(input) / frame_size )
  else:
    num_frames = int(num_frames)
  
  # Call the animation iterator (FuncAnimation)
  fftAnim = animation.FuncAnimation(fftFig, fftAnimFunc, frames=num_frames, interval=frame_period_in_ms, blit=True)
  
  return fftAnim

In [65]:
clip_dur = 20
fftAnimation = fftAnim(x_r, num_frames=30*clip_dur, sample_rate=fs44, frame_size=1024)
fftAnimation  # Display the animation
              # Again, we could call fftAnim() without a return value, but we'll want to use the output below

In [61]:
def animWithSound(anim_frames, audio_data, sample_rate=44100):
  # This is just a hack to create unique filenames (based on the current timestamp)
  dt_suffix = str( int( np.datetime64('now').astype(np.timedelta64) / np.timedelta64(1, 's') ) ) # Current date/time in seconds
  anim_filename = 'temp_anim_' + dt_suffix + '.mp4'
  audio_filename = 'temp_audio_' + dt_suffix + '.wav'
  output_filename = 'temp+sound_' + dt_suffix + '.mp4'
  
  anim_frames.save(anim_filename)
  sf.write(audio_filename, audio_data, sample_rate)
  !ffmpeg -i $anim_filename -i $audio_filename -map 0 -map 1:a -c:v copy -shortest $output_filename -hide_banner -loglevel error
  return output_filename  # Return the filename of the temp output

In [66]:
fftAnim_file = animWithSound(fftAnimation, x_r[:fs44*clip_dur], fs44)
ipd.Video(fftAnim_file, embed=True)